# Importing Libraries and Helper Classes and Functions

In [1]:
import sys
import os
import json
import numpy as np
from tqdm import tqdm
import torch
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
from utils_data import  generateTargetDataBuySide, generateTargetDataSellSide, getTechnicalIndicators, normalize_dataframe_with_mean_std
from utils_data import UpstoxStockDataFetcher
import pandas as pd

# Loading Data

In [2]:
fetcher = UpstoxStockDataFetcher()
stock_symbol = "SUZLON"
start_date = "2025-06-15"
end_date = "2025-06-18"
df = fetcher.get_stock_data(stock_symbol, start_date, end_date)
df


,time,open,high,low,close,volume
0,2025-06-16 09:15:00+05:30,64.50,65.05,64.45,65.05,1860104
1,2025-06-16 09:16:00+05:30,65.07,65.30,64.86,64.97,1193739
2,2025-06-16 09:17:00+05:30,64.96,65.07,64.88,64.92,571128
3,2025-06-16 09:18:00+05:30,64.95,65.01,64.60,64.66,772514
4,2025-06-16 09:19:00+05:30,64.66,64.69,64.38,64.57,1089367
...,...,...,...,...,...,...
1120,2025-06-18 15:25:00+05:30,63.96,63.96,63.91,63.94,118175
1121,2025-06-18 15:26:00+05:30,63.94,63.94,63.89,63.91,262339
1122,2025-06-18 15:27:00+05:30,63.89,63.92,63.86,63.89,264268
1123,2025-06-18 15:28:00+05:30,63.89,63.91,63.88,63.89,254881


In [4]:
df_today = pd.read_csv(r"C:\Users\srija\Assignment\Trading\json_files\suzlon_2025-06-19.csv")
df_today = df_today.drop_duplicates()
df_today['time'] = df_today['time'].astype(str) + "+05:30"
df_today['time'] = pd.to_datetime(df_today['time'])
df = pd.concat([df, df_today], ignore_index=True)
df


,time,open,high,low,close,volume
0,2025-06-16 09:15:00+05:30,64.50,65.05,64.45,65.05,1860104
1,2025-06-16 09:16:00+05:30,65.07,65.30,64.86,64.97,1193739
2,2025-06-16 09:17:00+05:30,64.96,65.07,64.88,64.92,571128
3,2025-06-16 09:18:00+05:30,64.95,65.01,64.60,64.66,772514
4,2025-06-16 09:19:00+05:30,64.66,64.69,64.38,64.57,1089367
...,...,...,...,...,...,...
1465,2025-06-19 15:02:00+05:30,62.30,62.34,62.26,62.34,239907
1466,2025-06-19 15:03:00+05:30,62.31,62.38,62.28,62.38,195570
1467,2025-06-19 15:05:00+05:30,62.39,62.42,62.37,62.40,137269
1468,2025-06-19 15:06:00+05:30,62.38,62.58,62.37,62.48,1174016


In [5]:
df = getTechnicalIndicators(df)
target_buy = generateTargetDataBuySide(df,1.005,0.99)
target_sell = generateTargetDataSellSide(df,0.995,1.01)

Processing Sell Side Data: 100%|██████████| 1443/1443 [00:29<00:00, 48.96it/s] 


# Data Analysis

In [6]:
count1 = len(target_buy[target_buy['action'].isin(['End of Day'])])
count2 = len(target_buy[target_buy['action'].isin(['Target Hit'])])
count3 = len(target_buy[target_buy['action'].isin(['Stop Loss Hit'])])

print(f"End of Day: {count1}")
print(f"Target Hit: {count2}")
print(f"Stop Loss Hit: {count3}")

End of Day: 362
Target Hit: 714
Stop Loss Hit: 367


In [7]:
count4 = len(target_sell[target_sell['action'].isin(['End of Day'])])
count5 = len(target_sell[target_sell['action'].isin(['Target Hit'])])
count6 = len(target_sell[target_sell['action'].isin(['Stop Loss Hit'])])

print(f"End of Day: {count4}")
print(f"Target Hit: {count5}")
print(f"Stop Loss Hit: {count6}")

End of Day: 414
Target Hit: 740
Stop Loss Hit: 289


# Normalizing and Storing Params

In [8]:
df_normalized, norm_param = normalize_dataframe_with_mean_std(df)

# Storing Norm parameters to be used later for inference
with open("C:/Users/srija/Assignment/Trading/json_files/suzlon_14_june_norm_params.json", "w") as f:
    json.dump(norm_param, f)
df_normalized

c:\Users\srija\Assignment\Trading\utils_data.py:210: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normalized_df[col].iloc[i] = normalized_df[col].iloc[i].hour*60 + normalized_df[col].iloc[i].minute-540
c:\Users\srija\Assignment\Trading\utils_data.py:210: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '42' has dtype incompatible with datetime64[ns, UTC+05:30], please explicitly cast to a compatible dtype first.
  normalized_df[col].iloc[i] = normalized_df[col].iloc[i].hour*60 + normalized_df[col].iloc[i].minute-540


,time,open,high,low,close,volume,MA50,RSI,MACD,BB_upper,BB_lower,ADX,CCI,ATR,ROC,OBV
0,42,63.42,63.54,63.40,63.54,0.148690,-0.311075,-2.278575,-5.195724,0.074711,-1.269002,2.170878,-0.438213,4.368766,-1.354629,-3.515543
1,43,63.52,63.73,63.46,63.73,-0.138830,-0.367216,-1.500105,-4.619538,-0.058979,-1.226689,1.851027,-0.126304,4.388066,0.752306,-3.458713
2,44,63.73,63.74,63.53,63.58,0.479474,-0.426334,-1.829013,-4.322464,-0.193186,-1.191916,1.548778,-0.120591,4.294710,0.319508,-3.574071
3,45,63.56,63.59,63.35,63.45,1.386853,-0.488855,-2.091144,-4.219633,-0.316810,-1.170754,1.352010,-0.452894,4.263660,-0.222588,-3.775320
4,46,63.45,63.45,63.31,63.35,0.771457,-0.544571,-2.280925,-4.228426,-0.425898,-1.163557,1.186530,-0.689508,4.049368,-0.168742,-3.918317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1438,362,62.30,62.34,62.26,62.34,0.297290,-2.166283,-0.527711,-0.627366,-2.308127,-2.035365,-1.531991,-1.902999,0.127486,-0.667989,-0.508280
1439,363,62.31,62.38,62.28,62.38,0.105736,-2.179468,-0.284715,-0.610843,-2.309199,-2.042373,-1.500356,-1.119902,0.134450,-0.006663,-0.428299
1440,365,62.39,62.42,62.37,62.40,-0.146149,-2.190951,-0.164639,-0.561454,-2.309861,-2.045181,-1.534483,-0.328497,0.048186,-0.172048,-0.372161
1441,366,62.38,62.58,62.37,62.48,4.333034,-2.194354,0.294862,-0.403414,-2.308203,-2.045626,-1.476509,0.820611,0.264821,0.213926,0.107969


# Loading Models

In [9]:
from Models.DQN import DQN,DQNAgent
from trading_environment import StockTradingEnv

policy_net = DQN(16, 3)
target_net = DQN(16, 3)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

DQN(
  (fc1): Linear(in_features=16, out_features=1440, bias=True)
  (fc2): Linear(in_features=1440, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=64, bias=True)
  (fc7): Linear(in_features=64, out_features=32, bias=True)
  (fc8): Linear(in_features=32, out_features=3, bias=True)
)

In [10]:
def get_state(df, current_step):
    row = df.iloc[current_step]
    state = np.array([
        row['time'],
        row['open'],
        row['high'],
        row['low'],
        row['close'],
        row['volume'],
        row['MA50'],
        row['RSI'],
        row['MACD'],
        row['BB_upper'],
        row['BB_lower'],
        row['ADX'],
        row['CCI'],
        row['ATR'],
        row['ROC'],
        row['OBV']
    ], dtype=np.float32)
    return state


In [14]:
save_folder = r"C:\Users\srija\Assignment\Trading\Models\trained_models\suzlon_14_june"
model_path = r"C:\Users\srija\Assignment\Trading\Models\trained_models\suzlon_14_june\suzlon_14_june_1496.pth"
policy_net.load_state_dict(torch.load(model_path))
policy_net.train()
env = StockTradingEnv(df_normalized)
agent = DQNAgent(env, policy_net, target_net)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
policy_net.to(device)
target_net.to(device)

DQN(
  (fc1): Linear(in_features=16, out_features=1440, bias=True)
  (fc2): Linear(in_features=1440, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=64, bias=True)
  (fc7): Linear(in_features=64, out_features=32, bias=True)
  (fc8): Linear(in_features=32, out_features=3, bias=True)
)

# Defining Reward Function

In [15]:
def calculate_optimized_scalping_reward(delay, action_type, success_base_reward=1500, 
                                     failure_base_penalty=1000, min_delay_threshold=60, 
                                     max_reward=2500, decay_rate=0.3,
                                     opportunity_cost_factor=0.2,
                                     missed_opp_multiplier=2.0,
                                     consecutive_successes=0, 
                                     consecutive_success_bonus=0.15):
    """
    Comprehensive reward function optimized for scalping.
    """
    delay = delay/60
    if action_type == 'success':
        # Delay-dependent base reward scaling
        if delay <= min_delay_threshold:
            base_reward = max_reward - (max_reward - success_base_reward) * (delay / min_delay_threshold)
        else:
            base_reward = success_base_reward
        
        # Apply exponential decay
        reward = base_reward * np.exp(-decay_rate * delay)
        
        # Apply opportunity cost
        opportunity_cost = opportunity_cost_factor * delay * success_base_reward
        opportunity_cost = min(opportunity_cost, reward * 0.8)
        reward = reward - opportunity_cost
        
        # Apply sequential bonus
        if consecutive_successes > 0:
            sequential_bonus = reward * (consecutive_success_bonus * consecutive_successes)
            reward += sequential_bonus
        
        return reward
    
    elif action_type == 'failure':
        # Standard penalty with exponential decay
        penalty = -failure_base_penalty * np.exp(-decay_rate * delay)
        
        # Add opportunity cost to penalty
        opportunity_cost = opportunity_cost_factor * delay * failure_base_penalty
        penalty = penalty - opportunity_cost
        
        return penalty
    
    elif action_type == 'missed_opportunity':
        # Enhanced penalty for missed opportunities
        missed_penalty = -failure_base_penalty * missed_opp_multiplier * np.exp(-decay_rate * delay)
        return missed_penalty
    
    elif action_type == 'no_action':
        # Reward for correctly staying out of the market
        return 100


# Training the Model 

In [ ]:
num_episodes = 3000

for episode in range(1500,num_episodes):
    # state = env.reset()
    total_reward = 0
    number_trans = 0
    wins =0
    lose = 0
    defeat =0
    consecutive_success = 0
    pbar = tqdm(total = len(df_normalized))
    step =0
    next_step = 0

    while step<len(df_normalized):
    
        
        state = get_state(df_normalized,step)
        action = agent.select_action(state)
        done = False
        # print(action)
       
        if action ==1: ## BUY
            next_state = target_buy.iloc[step]
            
            next_state_index = next_state["next_state_index"]
            
            next_state2 = df_normalized.iloc[next_state_index].copy()

            reward = 0
            if(next_state['action']=="Target Hit"):
                wins +=1
                consecutive_success+=1
                # reward = 1000/(target_buy.iloc[step]['delay']+1)
                reward = calculate_optimized_scalping_reward(delay=target_buy.iloc[step]['delay'],action_type="success",consecutive_successes=consecutive_success)
                
              
            elif next_state['action']=="Stop Loss Hit":
                defeat +=1
                consecutive_success=0
                # reward = -1000/(target_buy.iloc[step]['delay']+1)
                reward = calculate_optimized_scalping_reward(delay=target_buy.iloc[step]['delay'],action_type="failure",consecutive_successes=consecutive_success)
                
                
            elif next_state['action']=="End of Day":
                lose+=1
                consecutive_success=0
                done = True
                # reward = -50
                reward = calculate_optimized_scalping_reward(delay=target_buy.iloc[step]['delay'],action_type="failure",consecutive_successes=consecutive_success)
                
               
            reward = float(reward)  # Convert to scalar float
            
            next_state2 = np.array(next_state2.values, dtype=np.float32)
            agent.store_transition(state, action, reward, next_state2, done)
            agent.update_policy()
            number_trans +=1
            next_step = next_state_index+1
        
        if action==2: ## Sell  short
            next_state = target_sell.iloc[step]
            next_state_index = next_state["next_state_index"]
            
            next_state2 = df_normalized.iloc[next_state_index].copy()
            reward = 0
            if(next_state['action']=="Target Hit"):
                wins +=1
                consecutive_success+=1
                # reward = 1000/(target_sell.iloc[step]['delay']+1)
                reward = calculate_optimized_scalping_reward(delay=target_sell.iloc[step]['delay'],action_type="success",consecutive_successes=consecutive_success)
                
               
            elif next_state['action']=="Stop Loss Hit":
                consecutive_success=0
                defeat +=1
                # reward = -1000/(target_sell.iloc[step]['delay']+1)
                reward = calculate_optimized_scalping_reward(delay=target_sell.iloc[step]['delay'],action_type="failure",consecutive_successes=consecutive_success)
                
               
            elif next_state['action']=="End of Day":
                consecutive_success=0
                lose+=1
                done =True
                # reward = -50
                reward = calculate_optimized_scalping_reward(delay=target_sell.iloc[step]['delay'],action_type="failure",consecutive_successes=consecutive_success)
                
                
            reward = float(reward)  # Convert to scalar float
           
            next_state2 = np.array(next_state2.values, dtype=np.float32)
            
            agent.store_transition(state, action, reward, next_state2, done)
            agent.update_policy()
            number_trans +=1
            next_step = next_state_index+1
        if action ==0:
            buy_side = target_buy.iloc[step].copy()
            sell_side = target_sell.iloc[step].copy()
            if buy_side['action']=="Target Hit":
                # reward = -1000/(target_buy.iloc[step]['delay']+1)
                reward = calculate_optimized_scalping_reward(delay=target_buy.iloc[step]['delay'],action_type="missed_opportunity",consecutive_successes=consecutive_success)     
            elif sell_side['action']=="Target Hit":
                # reward = -1000/(target_sell.iloc[step]['delay']+1)
                reward = calculate_optimized_scalping_reward(delay=target_sell.iloc[step]['delay'],action_type="missed_opportunity",consecutive_successes=consecutive_success)
            else:
                reward = 100
            if step+1 < len(df_normalized):
               
                next_state = get_state(df_normalized,step+1)
                
                reward = float(reward)  # Convert to scalar float
                agent.store_transition(state, action, reward, next_state, done)
                agent.update_policy()
            else:
                done = True
                next_state = get_state(df_normalized,-1)
                reward = float(reward)  # Convert to scalar float
                agent.store_transition(state, action, reward, next_state, done)
                agent.update_policy()
            next_step = step+1
            
        pbar.update(next_step - step)
        step = next_step
    pbar.close()
                    
    
    # Update the target network
    if episode % 5 == 0:
        target_net.load_state_dict(policy_net.state_dict())
        model_save_path = os.path.join(save_folder, f'suzlon_14_june_{episode+1}.pth')
        print(f'Episode {episode + 1}')
        print(f'Number of transcations: {number_trans}, Wins: {wins}, Lose: {lose}, Defeat: {defeat} ')

        # Save the model state_dict
        torch.save(policy_net.state_dict(), model_save_path)       


100%|██████████| 1443/1443 [00:00<00:00, 85151.25it/s]


Episode 1501
Number of transcations: 26, Wins: 15, Lose: 4, Defeat: 7 


100%|██████████| 1443/1443 [00:00<00:00, 110009.28it/s]


Episode 1506
Number of transcations: 27, Wins: 15, Lose: 4, Defeat: 8 


100%|██████████| 1443/1443 [00:00<00:00, 102132.65it/s]


...

100%|██████████| 1443/1443 [00:09<00:00, 154.17it/s]


Episode 2866
Number of transcations: 39, Wins: 28, Lose: 1, Defeat: 10 


 80%|████████  | 1155/1443 [00:06<00:01, 153.18it/s]

KeyboardInterrupt: 

 82%|████████▏ | 1181/1443 [00:17<00:01, 153.18it/s]